# Manipulation

## CSV

### Imports

In [ ]:
import csv

### Functions

#### Read

In [ ]:
def csv_read_file(path):
    ''' Lê o arquivo csv e retorna seus dados em uma lista '''
    data_file = []
    with open(path, 'r', encoding='utf-8') as file:
        read = csv.reader(file)
        for row in read:
            data_file.append(row)
    return data_file


#### Write

In [ ]:
def csv_write_file(path, collection):
    ''' Escreve os dados de uma coleção no arquivo CSV linha a linha '''
    with open(path, 'w', encoding='utf-8', newline='') as file:
        write = csv.writer(file)
        for row in collection:
            write.writerow(row)
    return 'File written with success!'

#### Create file CSV

- Para testar as funções, criei uma arquivo CSV com alguns nomes aleatórios que peguei no site https://rogertakemiya.com.br/gerador-de-nomes-de-pessoas/ "Lista com alguns dos nomes utilizados no Estados Unidos".

- O arquivo CSV já tinha dois campos onde estavam os nomes e o outro  estava vazio. Então eu adicionei um cabeçalho manualmente, criando os campos que iria utitilizar e para não ter que preencher os 50 registros um por um, fiz um programa que iria preencher os campos com 5 possibilidades escolhidas aleatóriamente de seu respectivo campo.

In [ ]:
import random

delivery = ['2022-02-12','2022-02-16','2022-02-08','2022-02-22','2022-02-28']
register = ['2022-02-02','2022-02-05','2022-01-23','2022-02-01','2022-01-28']
type_order = ['design','paisagem','logo','personagem','caricatura']

data = csv_read_file("../Data/test_copy.csv")
for row in data[1:]:
    row[1] = random.choice(type_order)
    row.append(random.choice(delivery))
    row.append(random.choice(register))
csv_write_file('../Data/final_test.csv', data)

## SQLite3

### Imports

In [ ]:
import sqlite3
import os
from datetime import datetime

### Functions

#### Dates

- A Data de registro usa a data do dia do registro, mas o SQLite3 não tem um tipo especifico para guardar datas, então se usa o tipo TEXT.
- Como boa pratica, as datas no banco de dados são guardados no formato AAAA-mm-dd, por causa do criterio de ordenação que existe no banco de dados. Assim as datas podem ser ordenadas pelo "ano", "mes" e então "dias", o que deixa tudo mais organizado

In [ ]:
def date_now():
    """ Captura a data  do dia atual e a retorna 
        como string no formato AAAA-mm-dd """
    date_now = datetime.today().strftime('%Y-%m-%d')
    return str(date_now)

def date_to_display(date):
    """ Converte o formato de uma data em string
        de AAAA-mm-dd para dd/mm/AAAA """
    date = date.split('-')[::-1]
    date = '/'.join(date)
    return date

def date_to_sqlite(date):
    """ Converte o formato de uma data em string
        de dd/mm/AAAA para AAAA-mm-dd"""
    date = date.split('/')[::-1]
    date = '-'.join(date)
    return date
    

#### Create and connect Database

In [ ]:
connection = sqlite3.connect('../Data/DB_Orders.db')
cursor = connection.cursor()

def close_db():
    """ Fecha a conexão com o banco de dados e o cursor """
    cursor.close()
    connection.close()

#### Create table

In [ ]:
sql_create = """CREATE TABLE IF NOT EXISTS Orders 
(id INTEGER PRIMARY KEY AUTOINCREMENT, 
client TEXT, 
type_order TEXT, 
date_delivery TEXT, 
date_register TEXT NOT NULL)"""

def fundb_create_table():
    """ Cria a tabela de pedidos, caso ela não exista """
    cursor.execute(sql_create)
    connection.commit()

- As strings de querys foram deixadas na parte de fora dos código para poderem ser reutilizadas sem ter que as escrever novamente, precisando apenas de algumas pequenas mudanças ou adições para seja lá o que for que a função irá consultar

#### Insert 

In [ ]:
sql_insert = """INSERT INTO Orders (client, type_order, date_delivery, date_register) 
VALUES (?,?,?,?)"""

def fundb_insert_collection(collection):
    """ Recebe uma coleção de registros e adiciona todos de uma vez no banco de dados """
    for row in collection:
        cursor.execute(sql_insert, row)
        connection.commit()
    
def fundb_insert_one(client, type_order, date_delivery):
    """ Recebe os dados para a criação de um registro e os adiciona no banco de dados, 
        já adicionando a data atual como a data de registro """
    date_register = date_to_sqlite(date_now())
    cursor.execute(sql_insert, (client, type_order, date_delivery, date_register))
    connection.commit()

#### Recover

In [ ]:
sql_select = """SELECT * FROM Orders"""

def fundb_recover_all():
    """ Regata todos os registros existentes no banco de dados 
        os retornando em um iterável """
    cursor.execute(sql_select)
    return cursor.fetchall()

def fundb_recover_specific(colum, specification):
    """ Regata um registro, buscado por uma coluna e valor especificos """
    sql_where = sql_select + f" WHERE {colum} = '{specification}'"
    cursor.execute(sql_where)
    record = cursor.fetchall()
    return record    

def fundb_find_id(name_client):
    """ Busca o ID do cliente especificado, retornando o valor do ID.
        Se o cliente tiver mais de um pedido, ele retorna o valor de 
        cada ID de pedido desse cliente em uma lista."""
    records = fundb_recover_specific('client', name_client)
    list_ids = []
    for rec in records:
        list_ids.append(rec[0])
    if len(list_ids) == 1:
        return list_ids[0]
    else:
        return list_ids

- Ao inves de fazer uma função para cada campo da coluna, fiz uma função que recebe a coluna que será usada como filtro e resgata todos os valores relacionados ao valor especificado.

#### Update

In [ ]:
sql_update = "UPDATE Orders SET "

def fundb_update_by_id(order_id, column, new_data):
    """ Recebe o ID de qual registro pretende atualizar, a coluna onde quer
        fazer a atualização e então o novo dado a ser introduzido """
    sql_where = sql_update + f"{column} = '{new_data}' WHERE id = {order_id}"
    cursor.execute(sql_where)
    connection.commit()
    return f'Update OK!'

#### Delete

In [ ]:
sql_delete = f"DELETE FROM Orders WHERE "

def fundb_delete_by_id(order_id):
    """ Deleta o registro do ID especificado """
    sql_where = sql_delete + f"id = {order_id}"
    cursor.execute(sql_delete)
    connection.commit()
    return f'Record deleted'

# Runing

## Useful

In [ ]:
def split_rec(rec):
    """ Divide cada parte do registro resgatado e passa como paramentro 
        para a função de impressão de registro """
    order_id = rec[0]
    client = rec[1]
    type_order = rec[2]
    delivery = date_to_display(rec[3])
    register = date_to_display(rec[4])
    print_recover(order_id, type_order, client, delivery, register)

def print_recover(order_id, type_order, client, delivery, register):
    """ Imprime os registros de um jeito mais organizado """
    display = '\n\n=============================\n'\
    f'ID: {order_id} \n'\
    f'Cliente: {client}\n'\
    f'Tipo do Pedido: {type_order}\n'\
    f'Data de Entrega: {delivery}\n'\
    f'Date de Registro: {register}\n'
    print(display)


### Displays

In [ ]:
display_menu = """
=========== Menu ============

0 - Sair
1 - Mostrar pedidos
2 - Inserir novo pedido
3 - Atualizar um novo pedido
4 - Apagar um pedido

"""
display_op1 = """
=========== Menu ============

1 - Mostrar todos os pedidos
2 - Pedido específico

"""
display_inserir = """
=========== Menu ============

1 - Inserir: 
"""
display_reset = """
=========== Menu ============

1 - Deseja resetar o banco de dados? 
"""

### Options

In [ ]:
def option_1():
    print(display_op1)
    option = int(input("Opção:  "))
    if option == 1:
        for rec in fundb_recover_all():
            split_rec(rec)
    elif option == 2:
        column = str(input("Buscar por: "))
        specific = str(input("Dado: "))
        if "date" in column:
            specific = date_to_sqlite(specific)
        for rec in fundb_recover_specific(column, specific):
            split_rec(rec)

def option_2():
    print(display_inserir)
    client = str(input("Cliente: "))
    type_order = str(input("Tipo do pedido: "))
    delivery = str(input("Data de entrega: "))
    register = date_now()
    fundb_insert_one(client, type_order, delivery)

def option_3():
    print(display_inserir)
    order_id = int(input("ID de quem quer atualizar: "))
    column = str(input("Campo a ser mudado: "))
    new_date = str(input("Atualização: "))
    if "date" in column:
        date_to_sqlite(new_date)
    fundb_update_by_id(order_id, column, new_date)

def option_4():
    print(display_inserir)
    order_id = int(input("ID de quem quer atualizar: "))
    ok = fundb_delete_by_id(order_id)
    return ok


def print_display_and_option(display):
    while True:
        print(display)
        option = int(input("Opção:  "))
        if option == 0:
            break
        elif option == 1:
            option_1()
            continue
        elif option == 2:
            option_2()
            continue
        elif option == 3:
            option_3()
            continue
        elif option == 4:
            option_4()
            continue
        elif option == 5:
            option_5()
        else:
            print("Opção invalida")
            continue

## Run

### Reset Database

- O reset serve para apagar o banco de dados e o recriar do zero, com os mesmo nomes porém com novas datas de entrega, registros e tipos de pedido, mas isto é apenas uma opção, podendo ser pulada cado queria.

In [ ]:
op = int(input("Reset Database? \n1 - Sim \n2 - Não \n"))
if op == 1:
    if os.path.exists('../Data/DB_Orders.db'):
        close_db()
        os.remove('../Data/DB_Orders.db')
    csv_path = '../Data/final_test.csv'
    data = csv_read_file(csv_path)[1:]
    connection = sqlite3.connect('../Data/DB_Orders.db')
    cursor = connection.cursor()
    fundb_create_table()
    fundb_insert_collection(data)

### Start

In [ ]:
print_display_and_option(display_menu)